In [22]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [23]:
tf.__version__

'2.0.0-alpha0'

In [24]:
keras.__version__

'2.2.4-tf'

In [25]:
model = keras.Sequential()
# 第一层是输入层，无需制定输入的shape，会自动匹配，也不需要activation？
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation="softmax"))

In [26]:
model.compile(optimizer="adam",
             loss="categorical_crossentropy",
             metrics=["accuracy"])

In [27]:
data = np.random.random((1000,32))
label = np.random.random((1000,10))
val_data = np.random.random((100, 32))
val_label = np.random.random((100, 10))

# 使用data/label训练模型

In [28]:
#history = model.fit(data, label, epochs=10, batch_size=100)

model.fit(data, label, epochs=10, batch_size=32,
          validation_data=(val_data, val_label))

Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 0s 313us/sample - loss: 12.2687 - accuracy: 0.0990 - val_loss: 13.1781 - val_accuracy: 0.0700
Epoch 2/10
1000/1000 [==============================] - 0s 103us/sample - loss: 24.4227 - accuracy: 0.1010 - val_loss: 25.1466 - val_accuracy: 0.1200
Epoch 3/10
1000/1000 [==============================] - 0s 106us/sample - loss: 76.4150 - accuracy: 0.1060 - val_loss: 50.6045 - val_accuracy: 0.1400
Epoch 4/10
1000/1000 [==============================] - 0s 113us/sample - loss: 180.3385 - accuracy: 0.1040 - val_loss: 95.1472 - val_accuracy: 0.1700
Epoch 5/10
1000/1000 [==============================] - 0s 106us/sample - loss: 331.4851 - accuracy: 0.0950 - val_loss: 149.3388 - val_accuracy: 0.1400
Epoch 6/10
1000/1000 [==============================] - 0s 111us/sample - loss: 537.0166 - accuracy: 0.0980 - val_loss: 194.3348 - val_accuracy: 0.0900
Epoch 7/10
1000/1000 [==========================

In [29]:
model.fit(data, label, epochs=10, batch_size=32,
          validation_data=(val_data, val_label))

Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 0s 127us/sample - loss: 2338.4718 - accuracy: 0.1050 - val_loss: 578.9163 - val_accuracy: 0.1700
Epoch 2/10
1000/1000 [==============================] - 0s 96us/sample - loss: 2855.4224 - accuracy: 0.1140 - val_loss: 590.1453 - val_accuracy: 0.1400
Epoch 3/10
1000/1000 [==============================] - 0s 117us/sample - loss: 3432.3975 - accuracy: 0.0870 - val_loss: 954.6440 - val_accuracy: 0.0300
Epoch 4/10
1000/1000 [==============================] - 0s 126us/sample - loss: 3996.7357 - accuracy: 0.0650 - val_loss: 592.6650 - val_accuracy: 0.1700
Epoch 5/10
1000/1000 [==============================] - 0s 125us/sample - loss: 4758.3293 - accuracy: 0.1000 - val_loss: 1117.8360 - val_accuracy: 0.0300
Epoch 6/10
1000/1000 [==============================] - 0s 108us/sample - loss: 5324.5494 - accuracy: 0.0930 - val_loss: 1105.9258 - val_accuracy: 0.1700
Epoch 7/10
1000/1000 [============

# 使用Dataset训练模型
直接使用data/label训练模型的时候，传入fit方法的data/label是完整数据集。使用Dataset训练模型时，传入fit的数据集必须经过batch/repeat处理。在使用Dataset训练模型时，所传入的steps_per_epoch参数意为：在每轮（epoch）训练中，每次喂入多少个批次的数据。因此，假设训练数据有1000行，每个批次100行，那么当steps_per_epoch=10时，一轮训练完毕正好使用完所有数据。

In [31]:
dataset = tf.data.Dataset.from_tensor_slices((data,label))
#print(dataset.output_shapes)
dataset = dataset.batch(100)
dataset = dataset.repeat()

#history = model.fit(dataset, epochs=10, steps_per_epoch=40)

In [34]:
val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_label))
val_dataset.batch(32).repeat()

# 执行会报错，只要加上validation_data这个部分？
model.fit(dataset,epochs=10,steps_per_epoch=20,
         validation_data=val_dataset,
         validation_steps=3)

W0504 21:32:10.429484 140385062082368 training_utils.py:1353] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.


Epoch 1/10
12/20 [=================>............] - ETA: 0s - loss: 10277.4464 - accuracy: 0.0942

InvalidArgumentError: Matrix size-incompatible: In[0]: [32,1], In[1]: [32,64]
	 [[{{node dense_3/MatMul}}]]
	 [[loss_1/output_1_loss/categorical_crossentropy/weighted_loss/broadcast_weights/assert_broadcastable/is_valid_shape/else/_23/has_valid_nonscalar_shape/then/_69/has_invalid_dims/concat/_14]] [Op:__inference_keras_scratch_graph_16711]

# 回调

In [ ]:
# 目前不能执行，报告 `CancelledError:  [Op:StatefulPartitionedCall]`
callbacks = [
  # Interrupt training if `val_loss` stops improving for over 2 epochs
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(data, label, batch_size=32, epochs=5, callbacks=callbacks,
          validation_data=(val_data, val_label))